# 제4 고지 : 신경망 만들기 
## STEP 37 : 텐서를 다루다.

지금까지는 변수는 주로 '스칼라'를 다뤘지만,머신러닝 프레임워크로 확장하기 위해서는 **벡터나 행렬 등의 텐서**를 다룰 수 있어야 한다.  
그래서 이번 단계에서는 텐서를 사용할때의 주의점과, 지금까지 구현한 DeZero 함수들이 텐서도 문제 없이 다룰 수 있는지 확인한다.

### 37.1 원소별 계산 (element-wise operation) 

현재까지는 다음과 같이 입/출력 모두 '스칼라' 라고 가정을 했다.

```python
import numpy as np 
import dezero.functions as F 
from dezero import Variable 

x = Variable(np.array(1.0)) # 입력 : 스칼라 
y = F.sin(x) # 출력 : 스칼라 
print(y) 
# variable(0.8414709848078965)
```

그렇다면 텐서일 경우는 어떻게 될지 다음 예제를 살펴보면  **입력/출력의 shape은 바뀌지 않은채로 원소별 계산**이 이뤄지는 것을 확인할 수 있다.


In [8]:
import sys

sys.path.append("..")
import numpy as np 
import dezero.functions as F 
from dezero import Variable 

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.sin(x)
print(f"sin(x) : {y}")

print("="*10)

x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
y = x + c 
print(f"x + c  : {y}")


sin(x) : variable([[ 0.84147098  0.90929743  0.14112001]
          [-0.7568025  -0.95892427 -0.2794155 ]])
x + c  : variable([[11 22 33]
          [44 55 66]])


### 37.2 텐서 사용 시의 역전파 
지금까지의 역전파 구현은 '스칼라'를 대상으로 했는데, '텐서'를 사용한 계산에 역전파를 적용하려면 어떻게 해야할까??  
사실 현재의 DeZero의 텐서의 역전파 계산은 **DeZero 함수에 '텐서'를 입력하면 텐서의 원소별 계산, 즉 '스칼라'로 계산하기 때문에 문제없이 작동**한다.

다음 예시를 통해 구체적으로 살펴보자.

```python
x = Variable(np.array([[1,2,3],[4,5,6]]))
c = Variable(np.array([[10,20,30],[40,50,60]]))
t = x + c 
y = F.sum(t)
```

위의 코드는 두 변수를 더한 다음 `sum` 함수로 모든 원소의 총합을 구한다.  (`sum()` 함수는  **step39** 에서 구현된다.)  
(참고로, 머신러닝 문제에서도 **텐서를 입력받아 스칼라를 출력** 하는 함수(`loss function`)를 설정하는 것이 일반적이다.)  

이를 계산그래프로 표현하면, 다음과 같다.
<p align='center'>
    <img src='../assets/%EA%B7%B8%EB%A6%BC%2037-1.png' align='center' width='50%'>
</p>

이제 위의 함수에 대한 역전파, 즉 **'마지막 출력이 스칼라인 계산 그래프'의 역전파**를 계산하면 다음과 같다.


```python
y.backward(retain_grad=True)
print(y.grad)
print(t.grad)
print(c.grad)
'''
variable(1)
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])
variable([[1 1 1]
          [1 1 1]])        
'''
```

여기서 주목할 것은 **기울기의 형상과 데이터(순전파 때의 데이터) 형상이 일치**한다는 것이다. 즉, `x.shape==x.grad.shape`, `c.shape==c.grad.shape`,`t.shape==t.grad.shape` 이다.



### 37.2 [보충] 텐서 사용 시의 역전파 

텐서를 사용했을 때의 역전파를 수식으로 살펴본다. (`numerator layout notation`을 따른다.)  
$\mathbf{y}=F(\mathbf{x}) , \mathbf{x,y}\in{\mathbb{R}^n}$ 함수가 있다고 가정하면, $\frac{\partial \mathbf{y}}{\partial \mathbf{x}}$는 다음과 같다.

$$
\frac{\partial \mathbf{y}}{\partial \mathbf{x}}=
\begin{bmatrix}
\frac{\partial y_1}{\partial x_1} & \frac{\partial y_1}{\partial x_2} & \dots & \frac{\partial y_1}{\partial x_n} \\
\frac{\partial y_2}{\partial x_1} & \frac{\partial y_2}{\partial x_2} & \dots & \frac{\partial y_2}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\ 
\frac{\partial y_n}{\partial x_1} & \frac{\partial y_n}{\partial x_2} & \dots & \frac{\partial y_n}{\partial x_n} 
\end{bmatrix}
$$

$\mathbf{y,x}$ 모두 벡터이므로 미분은 위와 같이 행렬의 형태가 되고, 이를 `Jacobian matrix` 이라 한다. 
참고로, $\frac{\partial y }{\partial \mathbf{x}} = \begin{bmatrix}\frac{\partial y}{\partial x_1} &\frac{\partial y}{\partial x_2} & \dots & \frac{\partial y}{\partial x_n}  \end{bmatrix}$ 은 $1\times n$ Jacobian matrix 으로 볼 수 있다.

다음으로 합성함수 $y=F(\mathbf{x})$ 가 $\mathbf{a}=A(\mathbf{x}),\mathbf{b}=B(\mathbf{a}),y=C(\mathbf{b})$ 라는 3개의 함수로 구성되어 있는 경우를 생각해보자.  
이때, $\mathbf{x,a,b}\in \mathbb{R}^n$ 이고, 최종 출력 $y$는 **스칼라** 인 경우를 가정하면, $\frac{\partial y}{\partial \mathbf{x}}$ 는 다음과 같다. 
<p align='center'>
    <img src='../assets/%EC%8B%9D%2037.1.png' align='center' width='50%'>
</p>

이어서 [식37.1]의 **행렬 곱을 계산하는 순서**를 두가지 방법 `forward` , `reverse` 로 나눠서 살펴본다. 

1. `forward` 방법 : 입력쪽에서 출력 쪽으로 계산 하는 방법 

![img](../assets/%EA%B7%B8%EB%A6%BC%2037-2.png)


2. `reverse` 방법 : 출력쪽에서 입력쪽으로 계산 하는 방법 

![img](../assets/%EA%B7%B8%EB%A6%BC%2037-3.png)

여기서 주목할 것은 `forward` 방법의 경우 중간 결과 $\frac{\partial \mathbf{b}}{\partial \mathbf{x}}$ 는 다시 **행렬**이 되지만, `reverse` 방법의 경우 중간 결과 $\frac{\partial y}{\partial \mathbf{a}}$ 는 **행렬이 아닌 n개의 행벡터**라는 것이다.  
즉, **`reverse` 모드의 계산 효율이 더 좋음**을 알 수 있다.

또 한가지 주목할 것은, **역전파 수행시 명시적으로 야코비 행렬을 구하여 행렬의 곱을 계산할 필요가 없다는 것**이다.  
예를들어, $\mathbf{a}=A(\mathbf{x})=sin(\mathbf{x})$ 가 **원소별 연산(element-wise operation)** 을 수행한 경우를 가정하면, 야코비 행렬은 다음과 같이 `diagonal matrix` 가 된다. 
$$
\begin{aligned}
\frac{\partial \mathbf{a}}{\partial \mathbf{x}}&=
\begin{bmatrix}
\frac{\partial a_1}{\partial x_1} & \frac{\partial a_1}{\partial x_2} & \dots & \frac{\partial a_1}{\partial x_n} \\
\frac{\partial a_2}{\partial x_1} & \frac{\partial a_2}{\partial x_2} & \dots & \frac{\partial a_2}{\partial x_n} \\
\vdots & \vdots & \ddots & \vdots \\ 
\frac{\partial a_n}{\partial x_1} & \frac{\partial a_n}{\partial x_2} & \dots & \frac{\partial a_n}{\partial x_n} 
\end{bmatrix}\\
&=
\begin{bmatrix}
\frac{\partial }{\partial x_1}sin(x_1) & \frac{\partial }{\partial x_2}sin(x_1) & \dots & \frac{\partial }{\partial x_n}sin(x_1) \\
\frac{\partial }{\partial x_1}sin(x_2) & \frac{\partial }{\partial x_2}sin(x_2) & \dots & \frac{\partial }{\partial x_n}sin(x_2) \\
\vdots & \vdots & \ddots & \vdots \\ 
\frac{\partial }{\partial x_1}sin(x_n) & \frac{\partial }{\partial x_2}sin(x_n) & \dots & \frac{\partial }{\partial x_n}sin(x_n) \\
\end{bmatrix}\\
&=
\begin{bmatrix}
\frac{\partial }{\partial x_1}sin(x_1) & 0 & \dots & 0 \\
0& \frac{\partial }{\partial x_2}sin(x_2)  & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\ 
0 & 0 & \dots & \frac{\partial }{\partial x_n}sin(x_n)  \\
\end{bmatrix}\\
&=
\begin{bmatrix}
\frac{\partial a_1}{\partial x_1} & 0 & \dots & 0 \\
0& \frac{\partial a_2}{\partial x_2}  & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\ 
0 & 0 & \dots & \frac{\partial a_n}{\partial x_n}\\
\end{bmatrix}
\end{aligned}
$$

그러므로, **원소별 연산(element-wise operation)에서의 최종 역전파의 결과**는 다음 수식과 같이 **야코비 행렬을 명시적으로 구하지 않고 미분을 원소별로 곱하여(원소별 연산) 구할 수 있다**는 것이다. 
$$
\begin{aligned}
\frac{\partial y}{\partial \mathbf{a}} \frac{\partial \mathbf{a}}{\partial \mathbf{x}} &= 
\begin{bmatrix}
\frac{\partial y}{\partial a_1} & \frac{\partial y}{\partial a_2} & \dots & \frac{\partial y}{\partial a_n}
\end{bmatrix}
\begin{bmatrix}
\frac{\partial a_1}{\partial x_1} & 0 & \dots & 0 \\
0& \frac{\partial a_2}{\partial x_2}  & \dots & 0 \\
\vdots & \vdots & \ddots & \vdots \\ 
0 & 0 & \dots & \frac{\partial a_n}{\partial x_n}\\
\end{bmatrix}\\
&=
\begin{bmatrix}
\frac{\partial y}{\partial a_1}\frac{\partial a_1}{\partial x_1} & \frac{\partial y}{\partial a_2}\frac{\partial a_2}{\partial x_2} & \dots & \frac{\partial y}{\partial a_n}\frac{\partial a_n}{\partial x_n}
\end{bmatrix}
\end{aligned}
$$